In [1]:
# https://stackoverflow.com/questions/29760935/how-to-get-vector-for-a-sentence-from-the-word2vec-of-tokens-in-sentence

In [25]:
import pandas as pd
import gensim

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split

In [27]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [28]:
data = pd.read_csv('80k_new_3.csv', sep = ';')

In [29]:
data.drop(data.columns[[2]], axis=1, inplace=True)

## 1 - не спам, 0 - спам

In [59]:
y = data['moderated']

In [60]:
X = data['text']

In [72]:
X = X.fillna('слово')
new_X = []

In [73]:
vocab = []

In [74]:
for i in X:
    arr = []
    sent = i.split()
    for word in sent:
        arr.append(word.strip('.!?)('))
        vocab.append(word.strip('.!?)('))
    new_X.append(arr)

In [88]:
vocab = list(set(vocab))

In [181]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.33, random_state=42)

In [89]:
X_train = np.array(X_train)

In [99]:
for i in X_train:
    print(i)
    break

['новогодние', 'пробки', '-', 'привет']


In [102]:
X_vecs = []
for i in X_train:
    v = []
    for w in i:
        v.append(vocab.index(w))
    X_vecs.append(v)

In [113]:
[9,9,8]+[0,0]

[9, 9, 8, 0, 0]

In [252]:
X_vecs_final = []

In [253]:
for i in X_vecs:
    if len(i) < 18:
        n_zer = 18-len(i)
        vector = i+[0]*n_zer
        vector = np.array(vector).reshape(1,-1)
        X_vecs_final.append(vector)
    else:
        i = np.array(i).reshape(1,-1)
        X_vecs_final.append(i)

In [255]:
len(X_vecs_final), len(X_vecs), len(y_train)

(30184, 30184, 30184)

In [182]:
y_train = np.array(y_train)
# y_tr = to_categorical(y_train)
# y_tr.shape

(30184, 2)

In [94]:
# y_test = list(y_test)

In [256]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding, Dense, GRU, Dropout

model = Sequential([
            Embedding(input_dim=len(vocab) + 1,
                output_dim=100,
                mask_zero=True,
                input_length=18,
                # embeddings_regularizer='l2'
            ),
            Dropout(0.2),
            GRU(100, 
                activation='tanh',
                # activity_regularizer='l2'
                ),
            Dropout(0.2),
            Dense(250, activation='tanh'),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [185]:
# y_tr = np.reshape(y_train, [-1, 1])

In [257]:
X_vecs_final[1].shape

(1, 18)

In [258]:
model.fit(X_vecs_final, y_train,
                    batch_size=32,
                    epochs=15,
                    verbose=1,
                    validation_split=0.2)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 arrays but instead got the following list of 30184 arrays: [array([[12401,  1705, 33912, 26135,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0]]), array([[43443, 19051, 26842, 41428,   271,     0,     ...

In [186]:
len(y_tr)

30184